In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/EzPC/gcn_gcn-lstm

Mounted at /content/gdrive
/content/gdrive/MyDrive/EzPC/gcn_gcn-lstm


In [34]:
!pip install stellargraph

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from stellargraph.core.utils import calculate_laplacian

#Run

In [ ]:
def data_preparation(datafile):
    #Put the file location
    df = pd.read_csv(datafile)
    
    #type casting
    df.pm1_0 = df.pm1_0.astype(float)
    df.pm2_5 = df.pm2_5.astype(float)
    df.pm10 = df.pm10.astype(float)
    df.lat = round(round(5*df.lat.astype(float),2)/5.0,3)
    df.long= round(round(5*df.long.astype(float),2)/5.0,3)

    # Ensuring Delhi region and removing outliers from data
    df = df[(df.lat.astype(int) == 28) &(df.long.astype(int) == 77)]
    df = df[(df.pm1_0<=1500) & (df.pm2_5<=1500) & (df.pm10<=1500) & (df.pm1_0>=20) & (df.pm2_5>=30) & (df.pm10>=30)]
    #df = df[(df.humidity<=60)&(df.humidity>=7)]
    # TODO: Add buffer and consider train-data for range
    # lat_range = {'min': df.lat.min(), 'max': df.lat.max()}
    # long_range = {'min': df.long.min(), 'max': df.long.max()}
    df['lat_grid'] = df.apply(lambda row: int((n_lat_grid-1)*(row.lat-lat_range['min'])/(lat_range['max']-lat_range['min'])), axis=1 )
    df['long_grid'] = df.apply(lambda row: int((n_long_grid-1)*(row.long-long_range['min'])/(long_range['max']-long_range['min'])), axis=1 )
    df['lat_grid'] = df['lat_grid'].astype(float).astype(int)
    df['long_grid'] = df['long_grid'].astype(float).astype(int)
    means = df[["lat_grid", "long_grid", "pm2_5"]].groupby(['lat_grid', 'long_grid']).mean().reset_index()['pm2_5'].values

    # rounding @120min
    df.dateTime = pd.to_datetime(df.dateTime)
    df.dateTime = df.dateTime.dt.round('120min')
    # use time as a feature as well
    df.dateTime = df.dateTime.dt.hour*60 + df.dateTime.dt.minute

    # only PM2_5
    df = df[['dateTime','lat_grid','long_grid','pm2_5']]
    df = df.pivot_table(index=['lat_grid','long_grid'], columns='dateTime', aggfunc='mean')['pm2_5']
    df = df.fillna(0)
    df['means'] = means
    return df

In [ ]:
def my_loss(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError()
    #mask = tf.sign(y_true)
    #y_pred = y_pred*(mask)
    loss= y_true-y_pred
    #loss = mse(tf.reshape(y_true,(-1,)),tf.reshape(y_pred,(-1,)))
#     print(loss)
    return loss

In [ ]:
def train_test_split(df, grid_points_indices, train_size):
    random.seed(10)
#     train_size = 0.8
    #train-data for range from total grid structure
    points_len = df.shape[0]
    test_size = int(points_len * (1-train_size))
    test_indices = random.sample(list(df.index), test_size)
    train_indices = list(set(df.index) - set(test_indices))

    train_data = df.loc[train_indices, :] 
    train_data = train_data.reindex(grid_points_indices,fill_value=0)
    train_data = np.array(train_data)

    test_data = df.loc[test_indices, :] 
    test_data = test_data.reindex(grid_points_indices,fill_value=0)
    test_data = np.array(test_data)
    return train_data, test_data

In [ ]:
def scale_data(train_data, test_data):
    max_pm = train_data.max()
    min_pm = train_data.min()
    train_scaled = (train_data - min_pm) / (max_pm - min_pm)
    test_scaled = (test_data - min_pm) / (max_pm - min_pm)
    return train_scaled, test_scaled

In [ ]:
def sequence_data_preparation(seq_len, train_data, test_data):
    trainX, trainY, testX, testY = [], [], [], []

    trainX.append(train_data[:, 0 : 0 + seq_len])
    trainY.append(train_data[:, -1])
    testX.append(test_data[:, 0 : 0 + seq_len])
    testY.append(test_data[:, -1])
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)

    return trainX, trainY, testX, testY

In [ ]:
def prepare_train_data(df, grid_points_indices, train_size):
    train_data, test_data = train_test_split(df, grid_points_indices, train_size)
    print("Train data: ", train_data.shape)
    print("Test data: ", test_data.shape)
    train_scaled, test_scaled = scale_data(train_data, test_data)
    trainX, trainY, testX, testY = sequence_data_preparation(
        seq_len, train_scaled, test_scaled
    )
    print(trainX.shape)
    print(trainY.shape)
    print(testX.shape)
    print(testY.shape)
    return trainX, trainY, testX, testY, train_data

In [ ]:
def train_model(trainX, trainY, testX, testY, train_data):
    cb = TimingCallback()

    history = model.fit(
            trainX,
            trainY,
            epochs = 100,
            batch_size = 40,
            shuffle=True,
            verbose=0,
            validation_data=[trainX, testY],
            callbacks=[cb]
        )

    ## Rescale values
    max_pm = train_data.max()
    min_pm = train_data.min()
    train_scaled = (train_data - min_pm) / (max_pm - min_pm)
    
    print(
        "Mean Train loss: ",
        sum(tf.sqrt(history.history["loss"]))*max_pm/len(history.history["loss"]),
        "\n Last Train loss: ",
        tf.sqrt(history.history["loss"][-1])*max_pm,
        "\n Mean Test loss:",
        sum(tf.sqrt(history.history["val_loss"]))*max_pm/len(history.history["val_loss"]),
        "\n Last Test loss:",
        tf.sqrt(history.history["val_loss"][-1])*max_pm,
    #     "\nTrain time ", traintime,
        "\n Train time callbacks", sum(cb.logs)
        
    )
    sg.utils.plot_history(history)

In [ ]:
def eval(model, trainX, testY):
    output = model.predict(trainX)
    # yhat = model.predict(testX)

    ## actual train and test values
    ## Rescale values
    max_pm = train_data.max()
    min_pm = train_data.min()
    print(max_pm)
    train_true = np.array((trainY * (max_pm - min_pm)) + min_pm)
    test_true = np.array((testY * (max_pm - min_pm)) + min_pm)

    ## Rescale model predicted values
    train_output = np.array((output * (max_pm - min_pm)) + min_pm)
    # test_rescpred = np.array((yhat * (max_pm - min_pm)) + min_pm)

    # # Masked predicted values
    # mask_train = tf.sign(trainY)
    mask_test = tf.sign(testY)
    # train_rescpred = train_rescpred*(mask_train)
    test_output = train_output*(mask_test)
    test_mse = my_loss(test_true, train_output)
    test_rmse = tf.sqrt(test_mse)
    print("Test RMSE: ", test_rmse)
    return test_output, test_true

In [ ]:
from scipy.spatial.distance import pdist, squareform

lat_range = {'min': 28.486, 'max': 28.72}
long_range = {'min': 77.1, 'max': 77.32}
n_lat_grid = 100
n_long_grid = 100
seq_len = 12

#adjacency matrix
grid_points = pd.read_csv('grid_points.csv')
n_grid_points = grid_points.shape[0] 
X = grid_points[["lat_grid", "long_grid"]].to_numpy()
f = lambda u, v: max(abs(u-v))<=1
adj = squareform(pdist(X, f))
eye = np.identity(n_grid_points)
adj = adj+eye
print(adj.shape) 

grid_points = grid_points.pivot_table(index=['lat_grid', 'long_grid'])
grid_points_indices = grid_points.index

(1989, 1989)


In [ ]:
print(adj.shape)

(1989, 1989)


#PyTorch Model
GCN layer x 2 -> LSTM layer -> Dense layer -> Sigmoid 

Define loss according to MSE 


In [ ]:
class GCN_Layer(nn.Module):
    def __init__(self, input_dimensions, output_dimensions, adj):
      super().__init__()

      self.d2=input_dimensions[1]
      self.d3=input_dimensions[2]
      if(type(output_dimensions)!=type(0)):
        self.d4=output_dimensions[2]
      else:
        self.d4=output_dimensions

      self.A=torch.nn.Parameter(adj)
      #self.A=torch.nn.Parameter(torch.randn(self.d2, self.d2))
      self.kernel=torch.nn.Parameter(torch.randn(self.d3, self.d4))
      self.bias=torch.nn.Parameter(torch.randn(self.d2, 1))
      
      self.relu=torch.nn.ReLU()
      #torch.nn.init.xavier_uniform_(A, gain=1.0)
      #torch.nn.init.xavier_uniform_(kernel, gain=1.0)

    def forward(self, inputs):
      
      a=torch.permute(inputs, (0, 2, 1))
      
      neighbours=torch.matmul(a, self.A)
      
      h_graph = torch.permute(neighbours, (0, 2, 1))
      output = torch.matmul(h_graph, self.kernel)
      
      if self.bias is not None:
        output += self.bias

      output = self.relu(output)
      return output

class GCN_LSTM(nn.Module):
    def __init__(self, A, model_input_dim, num_layers=1, lstm_units=4, hidden_dim=4):
      super().__init__()
      self.lstm_size=lstm_units
      self.hidden_dim=hidden_dim
      self.num_layers=num_layers
      self.initial_hidden= torch.randn(self.num_layers, 1, 4)
      self.initial_cell= torch.randn(self.num_layers, 1, 4)
      #Intialize the GCN_LSTM layer
      self.adj = torch.from_numpy(calculate_laplacian(A) if A is not None else None)

      adj_init = torch.zeros(1989, 1989)

      if self.adj is None:
        nn.init.zeros_(adj_init)

      self.A = self.adj

      self.gcn_layer1=GCN_Layer(model_input_dim, lstm_units, self.A)
      self.gcn_layer2=GCN_Layer(torch.Size([model_input_dim[0], model_input_dim[1], lstm_units]), lstm_units, self.A)
      
      self.lstm = nn.LSTM(input_size=1989, hidden_size=self.hidden_dim, num_layers=self.num_layers, dropout=0, batch_first=True)

      self.dense = torch.nn.Linear(hidden_dim, model_input_dim[1])
      self.activation = torch.nn.Sigmoid()

    def forward(self, inputs):
      output=self.gcn_layer1(inputs)
      output=self.gcn_layer2(output)
      outputgcn=torch.permute(output, [0, 2, 1])
      
      print(outputgcn)
      output=self.lstm(outputgcn, (self.initial_hidden, self.initial_cell))
      
      #last hidden state
      output=output[0][0][-1]
      print(output)

      output=self.dense(output)
      output=self.activation(output)

      return output

In [ ]:
inputs = torch.randn(1, 1989, 12)
labels = torch.randn(1989)

In [ ]:
num=4
tanh=torch.nn.Tanh()
sigmo=torch.nn.Sigmoid()
relu_act=torch.nn.ReLU()

In [ ]:
adj = adj.astype(np.float32)
model=GCN_LSTM(torch.from_numpy(adj), inputs.shape, 2)
x=model(inputs)
print(x)
print(x.shape)

tensor([[[0.0000, 0.0000, 1.0058,  ..., 0.0000, 0.2940, 0.6939],
         [0.0000, 0.0000, 0.0000,  ..., 1.2502, 1.2370, 1.9138],
         [0.0000, 0.9403, 2.5725,  ..., 2.8947, 0.4946, 1.5600],
         [0.0000, 1.0623, 3.3300,  ..., 0.0000, 0.0441, 0.7072]]],
       grad_fn=<PermuteBackward0>)
tensor([-0.0883, -0.1159, -0.1094,  0.0609], grad_fn=<SelectBackward0>)
tensor([0.4483, 0.5141, 0.5500,  ..., 0.5320, 0.5978, 0.5394],
       grad_fn=<SigmoidBackward0>)
torch.Size([1989])


In [ ]:
model_weights=[]
for i in model.parameters():
  model_weights.append(i)

#Forward

In [ ]:
#Modularizing fixed_gcn_lstm
def gcn_layers_forward(m_weights, inputs_gcn, num_gcn):
  inp=torch.clone(inputs_gcn)
  index=0
  fixed_layer_inputs, relu_outputs=[], []
  for i in range(num_gcn):
    a=torch.squeeze(inp)
    
    neighbours=torch.matmul(m_weights[index], a)
    fixed_layer_inputs.append(a)
    output = torch.matmul(neighbours, m_weights[index+1])
    
    fixed_layer_inputs.append(neighbours)
    output += m_weights[index+2]
    relu_outputs.append(output)
    inp = relu_act(output)
    index=index+3
  return inp, relu_outputs, fixed_layer_inputs, index

In [ ]:
sig=torch.nn.Sigmoid()
tan=torch.nn.Tanh()

In [ ]:
def lstm_forward(inputs_from_gcn, m_weights, num_lstm, index):
  caches=[]
  inputs_total=[]
  y=inputs_from_gcn
  
  inputs_total.append(inputs_from_gcn)
  
  for j in range(num_lstm):
    c=model.initial_cell[j]
    h=model.initial_hidden[j]
    cache=[]
    inputs_np=[]
    cache.append((h, c, 0, 0, 0, 0, 0))
    for i in range(4):
        inp=y[i]

        if(len(inp.shape)==1):
          inp=torch.unsqueeze(inp, 0)
          
        z = torch.matmul(inp, torch.transpose(model_weights[index], 0, 1))
        z += torch.matmul(torch.squeeze(h, dim=0), torch.transpose(model_weights[index+1], 0 , 1))
        z += model_weights[index+2] + model_weights[index+3]
        z0, z1, z2, z3 = torch.split(z, 4, dim=1)

        i = sig(z0)
        f = sig(z1)
        c = f * c + i * tan(z2)
        o = sig(z3)
        h = o * tan(c)

        inputs_np.append(h)
        cache.append((h, c, i, f, tan(z2), o, inp))
    index=index+4
    caches.append(cache)
    y=inputs_np
    #The last one won't be counted during backpropagation
    inputs_total.append(inputs_np)
  return h, caches, inputs_total, index

In [ ]:
output, _, _, ind=gcn_layers_forward(model_weights, inputs, 2)
y=torch.transpose(output, 1, 0)
h, caches, inputs_tot, ind=lstm_forward(y, model_weights, 2, ind)

In [ ]:
print(h)

tensor([[-0.0883, -0.1159, -0.1094,  0.0609]], grad_fn=<MulBackward0>)


In [ ]:
op=torch.matmul(torch.squeeze(h, dim=1), torch.transpose(model_weights[-2], 1, 0))
op+=model_weights[-1]
print(sig(op))

tensor([[0.4483, 0.5141, 0.5500,  ..., 0.5320, 0.5978, 0.5394]],
       grad_fn=<SigmoidBackward0>)


#Backward manual equations

In [ ]:
loss=(labels-x).sum()
loss.backward()

In [ ]:
def relu_derivative(ex):
  relu_deriv=[]
  x=ex.cpu().detach().numpy()
  for i in x:
    l=[]
    for j in i:
      if(j>0):
        l.append(1.0)
      else:
        l.append(0.0)
    relu_deriv.append(l)
  return torch.tensor(relu_deriv)

In [ ]:
#deriv=(labels-x)
deriv=torch.ones(1989)
deriv=-deriv*x*(1-x)
deriv=torch.unsqueeze(deriv, dim=1)
h=torch.unsqueeze(h, dim=0)

dL_dh=torch.matmul(torch.transpose(deriv, 1, 0), model_weights[-2])
deriv=torch.matmul(deriv, h)
deriv=torch.squeeze(deriv)

In [ ]:
model_weights[-2].grad

tensor([[ 0.0218,  0.0287,  0.0271, -0.0151],
        [ 0.0220,  0.0289,  0.0273, -0.0152],
        [ 0.0218,  0.0287,  0.0271, -0.0151],
        ...,
        [ 0.0220,  0.0289,  0.0272, -0.0152],
        [ 0.0212,  0.0279,  0.0263, -0.0146],
        [ 0.0219,  0.0288,  0.0272, -0.0151]])

In [ ]:
print(deriv)

tensor([[ 0.0218,  0.0287,  0.0271, -0.0151],
        [ 0.0220,  0.0289,  0.0273, -0.0152],
        [ 0.0218,  0.0287,  0.0271, -0.0151],
        ...,
        [ 0.0220,  0.0289,  0.0272, -0.0152],
        [ 0.0212,  0.0279,  0.0263, -0.0146],
        [ 0.0219,  0.0288,  0.0272, -0.0151]], grad_fn=<SqueezeBackward0>)


In [43]:
dh=dL_dh
dgates1=[]
dct_1=0
ft_1=0
dX1=[]
dH=[]
dH.append(dL_dh)
cache=caches[1]
for index in range(num):
  ind=num-index

  h, c, i, f, g, o, x=cache[ind]
  h=torch.squeeze(h, dim=0)
  c=torch.squeeze(c, dim=0)
  cprev=cache[ind-1][1]
  cprev=torch.squeeze(cprev, dim=0)

  dct=dh*o*(1-pow(tanh(c),2)) + dct_1*ft_1
  dct_1=dct
  ft_1=f

  dgt=dct*i*(1-pow(g,2))
  dit=dct*g*i*(1-i)
  dft=dct*cprev*f*(1-f)
  dot=dh*tanh(c)*o*(1-o)

  dgate=torch.cat((dit, dft, dgt, dot), 1)
  dXt=torch.matmul(dgate, model_weights[10])
  dX1.append(dXt)
  dh=torch.matmul(dgate, model_weights[11])
  dH.append(dh)
  dgates1.append(dgate)

dgates1.reverse()
dX1.reverse()
dH.reverse()

In [44]:
print(len(dX1))

4


In [53]:
#dh=dH[-1]*torch.matmul(dgates1[-1], model_weights[10])
dh=dX1[-1]
dgates=[]
dct_1=0
ft_1=0
dX=[]
cache=caches[0]
for index in range(num):
  ind=num-index

  h, c, i, f, g, o, x=cache[ind]
  h=torch.squeeze(h, dim=0)
  c=torch.squeeze(c, dim=0)
  cprev=cache[ind-1][1]
  cprev=torch.squeeze(cprev, dim=0)

  dct=dh*o*(1-pow(tanh(c),2)) + dct_1*ft_1
  dct_1=dct
  ft_1=f

  dgt=dct*i*(1-pow(g,2))
  dit=dct*g*i*(1-i)
  dft=dct*cprev*f*(1-f)
  dot=dh*tanh(c)*o*(1-o)

  dgate=torch.cat((dit, dft, dgt, dot), 1)
  #dX.append(dXt)
  try:
    dh=torch.matmul(dgate, model_weights[7]) + dX1[-2-index]
  except:
    print("Last")
  dgates.append(dgate)

dgates.reverse()
dX.reverse()

torch.Size([1, 16])
torch.Size([1, 16])
torch.Size([1, 16])
torch.Size([1, 16])
Last


In [ ]:
print(model_weights[-5].shape)

torch.Size([16, 4])


In [56]:
for i in model.parameters():
  print(i.shape)

torch.Size([1989, 1989])
torch.Size([12, 4])
torch.Size([1989, 1])
torch.Size([1989, 1989])
torch.Size([4, 4])
torch.Size([1989, 1])
torch.Size([16, 1989])
torch.Size([16, 4])
torch.Size([16])
torch.Size([16])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16])
torch.Size([16])
torch.Size([1989, 4])
torch.Size([1989])


In [58]:
dW=torch.zeros((1989, num*4))
dU=torch.zeros((num, num*4))
#Start the inputs from the second last input_tots (as input_tots contains one redundant gate)
y=inputs_tot[0]
print(len(y))
for i in range(num):
  xnext=torch.unsqueeze(torch.squeeze(y[i]), dim=1)
  hnext=torch.unsqueeze(torch.squeeze(cache[i][0]), dim=1)
  dW+=torch.matmul(xnext, dgates[i])
  dU+=torch.matmul(hnext, dgates[i])

4


In [63]:
print(model_weights[-10].grad)
print(torch.transpose(dW, 0, 1))

tensor([[ 0.0000e+00, -5.7564e-27,  1.1066e-21,  ..., -1.0596e-17,
         -1.0484e-17, -1.6220e-17],
        [ 0.0000e+00,  5.8544e-19, -2.2472e-16,  ...,  1.9021e-14,
          1.8752e-14,  2.8959e-14],
        [ 0.0000e+00,  6.0561e-18,  3.2049e-12,  ...,  3.0489e-06,
          3.0167e-06,  4.6673e-06],
        ...,
        [ 0.0000e+00,  3.5082e-15,  9.5976e-15,  ...,  9.5455e-10,
          9.4444e-10,  1.4612e-09],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00, -2.2613e-09, -6.1863e-09,  ..., -7.1870e-09,
         -1.4128e-09, -4.0971e-09]])
tensor([[ 0.0000e+00, -5.7564e-27,  1.1066e-21,  ..., -1.0596e-17,
         -1.0484e-17, -1.6220e-17],
        [ 0.0000e+00,  5.8543e-19, -2.2472e-16,  ...,  1.9021e-14,
          1.8752e-14,  2.8959e-14],
        [ 0.0000e+00,  6.0562e-18,  3.2050e-12,  ...,  3.0489e-06,
          3.0167e-06,  4.6673e-06],
        ...,
        [ 0.0000e+00,  3.5082e-15,  9.5975e-15

In [62]:
print(model_weights[-9].grad)
print(torch.transpose(dU, 0, 1))

tensor([[ 1.0283e-21, -3.9646e-21,  2.7910e-22,  5.2448e-22],
        [-2.1032e-16, -4.9796e-17, -5.7064e-17, -1.0727e-16],
        [ 2.9784e-12,  1.2223e-09,  7.8144e-13,  1.5190e-12],
        [-2.1936e-22, -1.3120e-15,  2.4515e-19,  5.3141e-24],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 8.6730e-06,  2.3677e-06,  2.3532e-06,  4.4235e-06],
        [-2.3287e-10, -6.3573e-11, -6.3183e-11, -1.1877e-10],
        [-8.8746e-17,  6.1096e-16, -2.3614e-17, -4.5263e-17],
        [ 2.9982e-21,  8.1850e-22,  8.1347e-22,  1.5292e-21],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.9398e-18, -1.8733e-11,  4.0854e-16,  1.0271e-19],
        [-1.3775e-19, -1.6686e-15,  3.5679e-16, -3.0868e-21],
        [ 1.7750e-12,  4.9930e-12, -1.9490e-13,  9.0519e-13],
        [ 4.0491e-20,  3.8243e-13, -1.0648e-17, -2.0767e-21],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 2.5917e-14, -8.3239e-14,  1.4947e-12,  1.3498e-14]])
tensor(

In [ ]:
tempdX = torch.squeeze(torch.stack((dX)))
tempdX = torch.transpose(tempdX, 1, 0)
print(tempdX.shape)

torch.Size([1989, 4])


In [ ]:
#Relu derivative
#relu_deriv=[1 else 0 for i in dX if (i>0)]print(tf.transpose(tf.squeeze(dy_dx))/dgcnw11)
relu_1=relu_derivative(relu_outputs[1])
drelu_1=tempdX*relu_1

#Weight 1 derivative
temp=fixed_layers_inputs[3]
dgcnw11=torch.matmul(torch.transpose(temp, 1, 0), drelu_1)

#Input 1 Derivative
dgcn_11=torch.matmul(drelu_1, torch.transpose(model_weights[4], 1, 0))
print(dgcn_11.shape)

print()

#Weight 2 derivative
temp=fixed_layers_inputs[2]
dgcnw12=torch.matmul(dgcn_11, torch.transpose(temp, 1, 0))
print(dgcnw12.shape)

#Input 2 derivative
dgcn_12=torch.matmul(torch.transpose(dgcn_11, 1, 0), model_weights[3]) #next weight set is 0, 2
print(dgcn_12.shape)

torch.Size([1989, 4])

torch.Size([1989, 1989])
torch.Size([4, 1989])


In [ ]:
drelu_2=torch.transpose(dgcn_12, 1, 0)*relu_derivative(relu_outputs[0]) #1989, 4

#Weight 1 derivative for the second layer
temp=fixed_layers_inputs[1] #1989, 12
dgcnw_21=torch.matmul(torch.transpose(drelu_2, 1, 0), temp)
dgcnw_21=torch.transpose(dgcnw_21, 1, 0)
print(dgcnw_21.shape) #12, 4


#Input 1 derivative for the second layer 
dgcn_21=torch.matmul(drelu_2, torch.transpose(model_weights[1], 1, 0))
print(dgcn_21.shape) #1989, 12

#Weight 2 derivative for the second layer
temp=fixed_layers_inputs[0] #1989, 12
dgcnw_22=torch.matmul(temp, torch.transpose(dgcn_21, 1, 0))
print(dgcnw_22)

torch.Size([12, 4])
torch.Size([1989, 12])
tensor([[ 1.7612e-04, -5.1107e-02, -8.4432e-02,  ...,  9.0496e-02,
         -6.1686e-01, -7.4245e-01],
        [ 1.9706e-03, -1.9223e-01, -3.0729e-01,  ...,  3.4333e-01,
         -5.9058e+00, -5.0830e+00],
        [-1.0532e-03,  6.1604e-02,  9.6172e-02,  ..., -1.1097e-01,
          1.7123e+00,  1.2825e+00],
        ...,
        [ 6.6052e-04, -8.3017e-02, -1.3506e-01,  ...,  1.4767e-01,
          9.5581e-02, -7.8040e-02],
        [-1.8453e-04,  1.0037e-02,  1.6688e-02,  ..., -1.8553e-02,
          1.3776e+00,  1.2478e+00],
        [ 2.3655e-04,  7.6297e-03,  1.2437e-02,  ..., -1.2471e-02,
         -2.5207e-01, -1.9913e-01]], grad_fn=<MmBackward0>)


In [ ]:
for name, i in model.named_parameters():
  print(name, i.shape)

gcn_layer1.A torch.Size([1989, 1989])
gcn_layer1.kernel torch.Size([12, 4])
gcn_layer1.bias torch.Size([1989, 1])
gcn_layer2.A torch.Size([1989, 1989])
gcn_layer2.kernel torch.Size([4, 4])
gcn_layer2.bias torch.Size([1989, 1])
lstm.weight_ih_l0 torch.Size([16, 1989])
lstm.weight_hh_l0 torch.Size([16, 4])
lstm.bias_ih_l0 torch.Size([16])
lstm.bias_hh_l0 torch.Size([16])
dense.weight torch.Size([1989, 4])
dense.bias torch.Size([1989])


In [ ]:
print(dW)
print(dU)

tensor([[ 4.2002e-02,  2.5625e-03, -2.7445e-05,  ...,  0.0000e+00,
          1.9070e-05, -8.1627e-08],
        [ 6.9039e-02,  7.9648e-03, -3.9230e-05,  ...,  0.0000e+00,
         -7.2163e-05, -1.1577e-07],
        [ 2.6551e-04,  1.2142e-04, -8.5891e-09,  ...,  0.0000e+00,
         -2.7817e-06,  9.6925e-16],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 5.2335e-02,  3.8095e-03, -3.3230e-05,  ...,  0.0000e+00,
          6.7532e-06, -9.8685e-08],
        [ 1.4177e-01, -1.9533e-01,  7.9495e-05,  ..., -7.5101e-01,
          8.3388e-03, -5.1476e-01]], grad_fn=<AddBackward0>)
tensor([[ 2.1752e-02,  2.1911e-01, -2.1628e-04,  1.8936e-02, -2.3464e-01,
          1.3047e-03,  1.3039e-03,  1.1243e-03,  0.0000e+00,  0.0000e+00,
         -6.4159e-08,  1.3305e-03, -2.5532e-01,  7.9065e-01, -8.7830e-03,
          5.4197e-01],
        [-2.4315e-02,  2.3918e-01, -2.4166e-04, -6.8558e-03, -2.8616e-01,
          1.2923e-03,  2.

In [ ]:
print(torch.transpose(dU, 1, 0))
print(model_weights[7].grad)

tensor([[-1.0521e-04,  3.5757e-04, -5.0933e-01,  1.6984e-04],
        [ 2.8990e-05,  2.1292e-03, -2.2362e-03,  3.9164e-03],
        [-1.3714e-01,  4.1827e-01,  2.6819e-01,  3.6792e-01],
        [-2.5542e-02,  7.7738e-02,  3.0848e-02,  6.8835e-02],
        [-9.6183e-04,  2.9316e-03,  1.3674e-03,  2.5833e-03],
        [-2.1268e-02,  6.4731e-02,  5.1634e-01,  5.6335e-02],
        [-5.4274e-03,  1.6597e-02,  2.1124e-03,  1.4487e-02],
        [ 1.0582e-01, -3.1042e-01,  1.9053e-01, -2.6476e-01],
        [ 3.0333e-10, -1.0158e-09,  6.7303e-05,  6.7572e-09],
        [-2.3456e-02,  1.0979e-01, -5.6073e-01, -9.5981e-02],
        [ 8.2539e-03, -2.5150e-02,  5.3782e-01, -2.2124e-02],
        [-1.0212e-02,  3.1121e-02,  1.4517e-02,  2.7435e-02],
        [-1.3909e-04,  4.6786e-04, -7.6458e-02,  2.9455e-04],
        [-1.6056e-02,  4.8915e-02, -1.0837e+00,  4.3047e-02],
        [ 1.4641e-04, -3.9861e-03, -1.3165e+00, -3.8034e-03],
        [-2.4713e-05,  4.3555e-05,  1.8067e-04,  1.9561e-05]],
       

#Extracing weights, inputs, initial hidden states in PyTorch

In [ ]:
count=0
for name, param in model.named_parameters():
  #np_arr = i.cpu().detach().numpy()
  print(name, param.shape)
  if(len(param.shape)==2):
    f=open('pytorch_gcn/'+name+'.txt', 'w')
    values=[]
    if(count==6 or count==7 or count==10):
      print('change', name)
      param=torch.transpose(param, 1, 0)
    param=param.cpu().detach().numpy()
    for k in param:
      for l in k:
        values.append(str(l)+'\n')
    for k in values:
      f.write(k)
    f.close()
  if(len(param.shape)==1):
    f=open('pytorch_gcn/'+name+'.txt', 'w')
    values=[]
    param=param.cpu().detach().numpy()
    for k in param:
        values.append(str(k)+'\n')
    for k in values:
      f.write(k)
    f.close()
  count+=1

gcn_layer1.A torch.Size([1989, 1989])
gcn_layer1.kernel torch.Size([12, 4])
gcn_layer1.bias torch.Size([1989, 1])
gcn_layer2.A torch.Size([1989, 1989])
gcn_layer2.kernel torch.Size([4, 4])
gcn_layer2.bias torch.Size([1989, 1])
lstm.weight_ih_l0 torch.Size([16, 1989])
change lstm.weight_ih_l0
lstm.weight_hh_l0 torch.Size([16, 4])
change lstm.weight_hh_l0
lstm.bias_ih_l0 torch.Size([16])
lstm.bias_hh_l0 torch.Size([16])
dense.weight torch.Size([1989, 4])
change dense.weight
dense.bias torch.Size([1989])


In [ ]:
bias_tensor=torch.zeros([16])
for name, param in model.named_parameters():
  if('lstm.bias' in name):
    bias_tensor+=param
    #print(param)
print(bias_tensor)
f=open('pytorch_gcn/'+'lstmbias'+'.txt', 'w')
values=[]
param=bias_tensor.cpu().detach().numpy()
for k in param:
    values.append(str(k)+'\n')
for k in values:
  f.write(k)
f.close()

tensor([-0.1013, -0.4464,  0.2253,  0.7108,  0.1682,  0.3344,  0.0110,  0.2105,
        -0.2275,  0.1678, -0.8396, -0.0029, -0.0024, -0.5787, -0.0526,  0.7909],
       grad_fn=<AddBackward0>)


In [ ]:
f=open('pytorch_gcn/'+'inputs.txt', 'w')
values=[]
param=inputs.cpu().detach().numpy()
for i in param:
  for k in i:
    for j in k:
      values.append(str(j)+'\n')
for i in values:
  f.write(i)
f.close()
print(len(values))

23868


In [ ]:
other_vals=[model.initial_hidden, model.initial_cell]
for ind in range(len(other_vals)):
  if(ind==0):
    f=open('pytorch_gcn/'+'hidden.txt', 'w')
  else:
    f=open('pytorch_gcn/'+'cell.txt', 'w')
  values=[]
  param=other_vals[ind].cpu().detach().numpy()
  for i in param:
    for k in i:
      for j in k:
        values.append(str(j)+'\n')
  for i in values:
    f.write(i)
  f.close()

In [ ]:
f=open('pytorch_gcn/'+'labels.txt', 'w')
values=[]
param=labels.cpu().detach().numpy()
for i in param:
  values.append(str(i)+'\n')
for i in values:
  f.write(i)
f.close()
print(len(values))

1989


In [ ]:
from google.colab import files
files.download('pytorch_gcn/labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>